Kaggle Id = AI20BTECH11002-AI20BTECH11024


In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vojeswithag","key":"3d555cd8422578447e103d4121e748bc"}'}

In [ ]:
 ! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c 'Foml-2021'

  0% 0.00/4.80M [00:00<?, ?B/s]
100% 4.80M/4.80M [00:00<00:00, 44.2MB/s]
  0% 0.00/6.97M [00:00<?, ?B/s]
100% 6.97M/6.97M [00:00<00:00, 64.0MB/s]


Installing the dataset from kaggle

In [ ]:
! mkdir train
! unzip train.csv.zip -d train

Archive:  train.csv.zip
  inflating: train/train.csv         


In [ ]:
! mkdir test
! unzip test.csv.zip -d test

Archive:  test.csv.zip
  inflating: test/test.csv           


Installing the dataset from kaggle, unzipping the files

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv('/content/train/train.csv') 
df_test = pd.read_csv('/content/test/test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Reading the files into dataframes


In [ ]:
df_train.head()

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,Municipality,Related Non-Motorist,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Non-Motorist Substance Abuse,Person ID,Injury Severity,Circumstance,Drivers License State,Vehicle ID,Vehicle Damage Extent,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location,Fault
0,MP060D,10196,Montgomery County Police,Injury Crash,06/05/2017 04:27:00 PM,Maryland (State),OLD HUNDRED RD,County,THURSTON RD,12305 SHOREFIELD RD,NaN,NaN,SINGLE VEHICLE,CLEAR,WET,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,466D8B-AC8C7,SUSPECTED MINOR INJURY,"N/A, WET",MD,7C2FEC-727F4,DESTROYED,TWELVE OCLOCK,TWELVE OCLOCK,PICKUP TRUCK,MOVING CONSTANT SPEED,South,Unknown,40,No,No,2005,GMC,TRUCK,NO MISUSE,39.263378,-77.344203,3.6383 7.4233,1
1,MP000X,20080,Montgomery County Police,Injury Crash,02/22/2020 10:00:00 AM,County,JANET RD,County,FLACK ST,NaN,NaN,NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,NaN,4028E4-C5384,NO APPARENT INJURY,"BACKUP DUE TO REGULAR CONGESTION, SLEET, HAIL,...",MD,67644F-64EF8,DISABLING,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,ACCELERATING,South,South,25,No,Yes,2018,FORD,FIESTA,NO MISUSE,39.068913,-77.063227,3.0862 7.324,1
2,MP070N,10103,Montgomery County Police,Property Damage Crash,06/11/2017 08:21:00 AM,Maryland (State),COLESVILLE RD,Maryland (State),STRUC #15082,NaN,NaN,NaN,SAME DIRECTION RIGHT TURN,CLEAR,DRY,UNKNOWN,OTHER,NaN,NaN,889DBD-D57FF,NO APPARENT INJURY,NaN,DC,30F7A9-92C98,SUPERFICIAL,SIX OCLOCK,SIX OCLOCK,TRANSIT BUS,PARKING,West,West,35,No,No,2016,NEW FLYER,TBU,NO MISUSE,38.993578,-77.032718,3.9583 7.3783,0
3,MP130Y,10208,Montgomery County Police,Injury Crash,05/10/2019 07:30:00 AM,County,PARKLAND DR,County,FRANKFORT DR,NaN,NaN,NaN,OTHER,CLOUDY,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,77155D-96AA7,POSSIBLE INJURY,NaN,MD,0F0C58-8E98F,FUNCTIONAL,TWELVE OCLOCK,TWO OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,South,South,25,No,No,2001,CHEV,SUBURBAN,OTHER,39.083480,-77.086933,"3.84,-70633",1
4,MP770Y,1091,Montgomery County Police,Injury Crash,08/02/2016 05:00:00 PM,Maryland (State),OLD GEORGETOWN RD,County,MCKINLEY ST,NaN,NaN,NaN,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,MEDICATION PRESENT,5E180B-42B0C,SUSPECTED MINOR INJURY,NaN,MD,4397C0-AEE83,DISABLING,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,SLOWING OR STOPPING,South,South,40,No,No,2013,KIA,RIO,NO MISUSE,38.996942,-77.109168,3.9917 7.0183,0


In [ ]:
df_test.head()

,Id,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,Municipality,Related Non-Motorist,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Non-Motorist Substance Abuse,Person ID,Injury Severity,Circumstance,Drivers License State,Vehicle ID,Vehicle Damage Extent,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location
0,0,MP810M,NaN,MONTGOMERY,Property Damage Crash,12/16/2015 05:42:00 PM,County,WHITTIER BLVD,Maryland (State),WILSON LA,NaN,NaN,NaN,OTHER,CLEAR,DRY,DARK LIGHTS ON,TRAFFIC SIGNAL,ALCOHOL CONTRIBUTED,NaN,F05A35-C3C98,NO APPARENT INJURY,NaN,MD,DA635F-C3055,FUNCTIONAL,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MAKING RIGHT TURN,North,North,25,No,No,2004,LEXUS,RX330,NO MISUSE,38.988440,-77.127668,"3.84,-71763"
1,1,MP2001,NaN,Montgomery County Police,Property Damage Crash,05/01/2016 07:25:00 PM,County,RAILROAD ST,Municipality,E DIAMOND AVE,NaN,GAITHERSBURG,NaN,SAME DIR REAR END,RAINING,WET,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,NaN,79C1D8-6A412,NO APPARENT INJURY,"RAIN, SNOW, WET",MD,DBC8E9-FC340,SUPERFICIAL,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,STOPPED IN TRAFFIC LANE,South,West,25,No,No,2001,MITSUBISHI,MIRAGE,NO MISUSE,39.094075,-77.205783,3.905 7.0733
2,2,MP6404,10125,Montgomery County Police,Property Damage Crash,03/16/2018 02:30:00 PM,Maryland (State),GEORGIA AVE,County,ARCOLA AVE,NaN,NaN,NaN,SAME DIRECTION SIDESWIPE,NaN,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,NaN,30EC44-418A7,NO APPARENT INJURY,NaN,MD,32DA16-845E9,NO DAMAGE,TWO OCLOCK,TWO OCLOCK,AMBULANCE/EMERGENCY,MOVING CONSTANT SPEED,North,North,35,No,No,2016,FRIEGHTLINER,AMBULANCE,NO MISUSE,39.047180,-77.052688,"3.41,-70283"
3,3,MP0405,10270,Montgomery County Police,Property Damage Crash,05/25/2018 05:00:00 PM,NaN,NaN,NaN,NaN,"19600 FREDERICK RD, PAWN SHOP PARKING LOT",NaN,NaN,SAME DIRECTION SIDESWIPE,CLEAR,NaN,DAYLIGHT,NO CONTROLS,NaN,NaN,C7FA23-4267E,NO APPARENT INJURY,NaN,MD,4DC569-AB8EA,DISABLING,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,ACCELERATING,West,West,25,No,No,2004,FORD,CN,NaN,39.178007,-77.238475,3.7067 7.345
4,4,MP090D,10374,Montgomery County Police,Property Damage Crash,09/23/2017 05:40:00 PM,Interstate (State),EISENHOWER MEMORIAL HWY,Maryland (State),MONTROSE RD,NaN,NaN,NaN,OTHER,CLEAR,DRY,DAYLIGHT,NO CONTROLS,UNKNOWN,NaN,B6023B-A34FF,NO APPARENT INJURY,NaN,NaN,B3BD7B-487E4,SUPERFICIAL,NINE OCLOCK,NINE OCLOCK,"MEDIUM/HEAVY TRUCKS 3 AXLES (OVER 10,000LBS (4...",CHANGING LANES,North,North,55,No,No,2008,UTIL,TL,UNKNOWN,39.056134,-77.154043,3.5143 7.5022


In [ ]:
Index = df_train.columns

In [ ]:
null_per = (df_train.isna().sum())*100/df_train.shape[0]
#print(null_per.sort_values(ascending= False))

In [ ]:
for column in enumerate(null_per):
    if (int(column[1])>70):
        col_in = column[0]
        df_train.drop(Index[col_in],axis='columns', inplace=True)
        df_test.drop(Index[col_in],axis='columns', inplace=True)
    elif (int(column[1]) != 0):
        col_in = column[0]
        df_train[Index[col_in]]  =  df_train[Index[col_in]].replace(np.NaN,df_train[Index[col_in]].mode()[0])  
        df_test[Index[col_in]]  =  df_test[Index[col_in]].replace(np.NaN,df_test[Index[col_in]].mode()[0])  

Columns having Null percent greater than 70 are removed as they don't contribute in training the model efficiently.
And the remaining null values are replaced by modes of corresponding column for columns of object type. There are no null values in columns of numeric type. 

In [ ]:
def convert_loc1(loc_1):
    loc_11 = []
    for x in enumerate(loc_1):
        l = list(x)
        if(l[1]<=0):
          l[1] = -l[1]
          for k in range(0,7):
              if((l[1]//10) != 0):
                 l[1] = l[1]/10  
        loc_11.append(l[1])         
    return loc_11      

df_train['Location'] = df_train['Location'].replace('\,','\n',regex=True)
df_train[['Location_0','Location_1']] = df_train.Location.str.split(expand=True).astype(float)
df_train['Location_1'] = convert_loc1(df_train['Location_1'])
df_train.head()

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Person ID,Injury Severity,Drivers License State,Vehicle ID,Vehicle Damage Extent,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location,Fault,Location_0,Location_1
0,MP060D,10196,Montgomery County Police,Injury Crash,06/05/2017 04:27:00 PM,Maryland (State),OLD HUNDRED RD,County,THURSTON RD,SINGLE VEHICLE,CLEAR,WET,DAYLIGHT,NO CONTROLS,NONE DETECTED,466D8B-AC8C7,SUSPECTED MINOR INJURY,MD,7C2FEC-727F4,DESTROYED,TWELVE OCLOCK,TWELVE OCLOCK,PICKUP TRUCK,MOVING CONSTANT SPEED,South,Unknown,40,No,No,2005,GMC,TRUCK,NO MISUSE,39.263378,-77.344203,3.6383 7.4233,1,3.6383,7.4233
1,MP000X,20080,Montgomery County Police,Injury Crash,02/22/2020 10:00:00 AM,County,JANET RD,County,FLACK ST,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,4028E4-C5384,NO APPARENT INJURY,MD,67644F-64EF8,DISABLING,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,ACCELERATING,South,South,25,No,Yes,2018,FORD,FIESTA,NO MISUSE,39.068913,-77.063227,3.0862 7.324,1,3.0862,7.3240
2,MP070N,10103,Montgomery County Police,Property Damage Crash,06/11/2017 08:21:00 AM,Maryland (State),COLESVILLE RD,Maryland (State),STRUC #15082,SAME DIRECTION RIGHT TURN,CLEAR,DRY,UNKNOWN,OTHER,NONE DETECTED,889DBD-D57FF,NO APPARENT INJURY,DC,30F7A9-92C98,SUPERFICIAL,SIX OCLOCK,SIX OCLOCK,TRANSIT BUS,PARKING,West,West,35,No,No,2016,NEW FLYER,TBU,NO MISUSE,38.993578,-77.032718,3.9583 7.3783,0,3.9583,7.3783
3,MP130Y,10208,Montgomery County Police,Injury Crash,05/10/2019 07:30:00 AM,County,PARKLAND DR,County,FRANKFORT DR,OTHER,CLOUDY,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,77155D-96AA7,POSSIBLE INJURY,MD,0F0C58-8E98F,FUNCTIONAL,TWELVE OCLOCK,TWO OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,South,South,25,No,No,2001,CHEV,SUBURBAN,OTHER,39.083480,-77.086933,3.84\n-70633,1,3.8400,7.0633
4,MP770Y,1091,Montgomery County Police,Injury Crash,08/02/2016 05:00:00 PM,Maryland (State),OLD GEORGETOWN RD,County,MCKINLEY ST,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,5E180B-42B0C,SUSPECTED MINOR INJURY,MD,4397C0-AEE83,DISABLING,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,SLOWING OR STOPPING,South,South,40,No,No,2013,KIA,RIO,NO MISUSE,38.996942,-77.109168,3.9917 7.0183,0,3.9917,7.0183


In [ ]:
df_test['Location'] = df_test['Location'].replace('\,','\n',regex=True)
df_test[['Location_0','Location_1']] = df_test.Location.str.split(expand=True).astype(float)
df_test['Location_1'] = convert_loc1(df_test['Location_1'])

As the column 'Location' of object type has two values, that column is divided into two columns namely 'Location_0' and 'Location_1' of float type.

In [ ]:
drop_columns = (['Report Number','Local Case Number','Crash Date/Time','Person ID','Vehicle ID','Location','Road Name','Cross-Street Name','Driverless Vehicle'])
for col in drop_columns:
    df_train.drop(col,axis='columns', inplace=True)
    df_test.drop(col,axis='columns', inplace=True)

The columns which are found to be unnecessary are dropped from both train and test dataset using for loop. They were found to have large number of unique values which may result in less accuracy of the model.

In [ ]:
Id = df_test.Id.values
df_test.drop('Id',axis='columns', inplace=True)

The column 'Id' is dropped from test dataset.
The Train dataset does not contain the column 'Id'.

In [ ]:
df_data = pd.concat([df_train.drop(columns=['Fault']),df_test], ignore_index=True)
object_columns = df_data.columns[df_data.dtypes == 'object']
print(len(object_columns))
print(object_columns)

23
Index(['Agency Name', 'ACRS Report Type', 'Route Type', 'Cross-Street Type',
       'Collision Type', 'Weather', 'Surface Condition', 'Light',
       'Traffic Control', 'Driver Substance Abuse', 'Injury Severity',
       'Drivers License State', 'Vehicle Damage Extent',
       'Vehicle First Impact Location', 'Vehicle Second Impact Location',
       'Vehicle Body Type', 'Vehicle Movement', 'Vehicle Continuing Dir',
       'Vehicle Going Dir', 'Parked Vehicle', 'Vehicle Make', 'Vehicle Model',
       'Equipment Problems'],
      dtype='object')


Concatenating the test and train(dropping the 'fault' column) dataset so that we have all unique entries of columns that are used in label encoding of the object type data.

In [ ]:
dict = {}
for col in object_columns:
      dict = {name:i for i,name in enumerate(df_data[col].unique())}
      def encode_name(name):
          return dict[name] 
      df_train[col] = df_train[col].apply(encode_name) 
      df_test[col] = df_test[col].apply(encode_name)  
      dict = {}   

Label Encoding of the train and test dataset.

In [ ]:
df_train.shape

(51490, 30)

In [ ]:
X_Train = df_train.drop(columns=['Fault']).to_numpy()
y_Train = df_train['Fault'].to_numpy()
X_Test = df_test.to_numpy()

In [ ]:
import xgboost
GBD2 = xgboost.XGBClassifier(learning_rate=0.05, n_estimators=300, max_depth=10)
GBD2.fit(X_Train,y_Train)
Feat_imp = GBD2.feature_importances_

Evaluating the feature importance of all the features while using xgboost. This improved the final accuracy because many features causing noise are removed.

In [ ]:
Feat_imp.shape

(29,)

In [ ]:
print(Feat_imp)

[0.01185377 0.01367265 0.01394407 0.01244714 0.1208243  0.01219583
 0.01377092 0.01466665 0.02815003 0.10682338 0.0170874  0.01476726
 0.01903745 0.20843855 0.04179024 0.01587502 0.09010056 0.01181724
 0.01173414 0.0207024  0.07454512 0.01276347 0.01209661 0.01182923
 0.03991255 0.01240441 0.01263176 0.01200683 0.01211097]


In [ ]:
X_Train.shape

(51490, 29)

In [ ]:
delete_feat_in = []
for i in range(0,29):
    if(float(Feat_imp[i])<=0.013):
      delete_feat_in.append(i)
print(delete_feat_in)      

[0, 3, 5, 17, 18, 21, 22, 23, 25, 26, 27, 28]


Gathering all the less important features so that they can be deleted.
Here the feature importance limit is taken as 0,013.

In [ ]:
X_Train =  np.delete(X_Train,delete_feat_in,axis=1)
X_Test =  np.delete(X_Test,delete_feat_in,axis=1)

Deleting those features.

In [ ]:
GBD = xgboost.XGBClassifier(learning_rate=0.05,min_samples_split=300,min_samples_leaf=50,max_depth=10,max_features='sqrt',subsample=0.8,random_state=10,n_estimators=300)
GBD = GBD.fit(X_Train,y_Train)

Using xgboost classifier for training our model with above selected hyperparameters.

In [ ]:
print("Accuracy over training set = ", GBD.score(X_Train,y_Train))

Accuracy over training set =  0.9307438337541271


Accuracy obtained over training set is 0.9307

In [ ]:
y_Predicted_T = GBD.predict(X_Test)

Predicting the outcome of the given test set.

In [ ]:
# Creating a new dataframe Predicted_prices_df which have the test Id , predicted_price
Predictions_df = pd.DataFrame()
Predictions_df['Id'] = Id

# Using this frame to get the predicitions of the new dataframe Predicted_prices_df
#predictions = GBD2.predict(df_test)
Predictions_df['Fault'] = pd.DataFrame(y_Predicted_T)

# Getting a csv file from the dataframe Predicted_prices_df
Predictions_df.to_csv('Predicted-data.csv', index = False)

Creating a .csv file for submitting all the predictions.

Gradient boosting is the model choosen which performs better than desicion trees.
It provides several hyperparameters so the model can be fitted flexibly with higher accuracy compared to other models